In [73]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import datetime
# fe = fm.FontEntry(
#     fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
#     name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
# fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
# plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
# plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim


# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

# import eli5
# from eli5.sklearn import PermutationImportance

In [131]:
# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
train_path = 'train.csv'
test_path  = 'test.csv'
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
rate = pd.read_csv("예금은행대출금리(신규취급액 기준)_주택담보대출.csv")

In [132]:
# Train data와 Test data shape은 아래와 같습니다.
print('Train data shape : ', train.shape, 'Test data shape : ', test.shape)

Train data shape :  (1118822, 52) Test data shape :  (9272, 51)


In [133]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1118822 entries, 0 to 1118821
Data columns (total 52 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   시군구                     1118822 non-null  object 
 1   번지                      1118597 non-null  object 
 2   본번                      1118747 non-null  float64
 3   부번                      1118747 non-null  float64
 4   아파트명                    1116696 non-null  object 
 5   전용면적(㎡)                 1118822 non-null  float64
 6   계약년월                    1118822 non-null  int64  
 7   계약일                     1118822 non-null  int64  
 8   층                       1118822 non-null  int64  
 9   건축년도                    1118822 non-null  int64  
 10  도로명                     1118822 non-null  object 
 11  해제사유발생일                 5983 non-null     float64
 12  등기신청일자                  1118822 non-null  object 
 13  거래유형                    1118822 non-null  object 
 14  중개

In [134]:
rate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    201 non-null    int64  
 1   rate    201 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 3.3 KB


In [135]:
# train/test 합치기 

train['is_test'] = 0
test['is_test'] = 1
concat = pd.concat([train, test])

In [136]:
# 대출금리 자료 합치기 
concat = pd.merge(concat, rate, left_on='계약년월', right_on='Date', how='left')
concat = concat.drop(columns=['Date'])

In [137]:
# 전용면적 단순화: 59->60, 84->85, 115->116
concat['전용면적(㎡)'] = concat['전용면적(㎡)'].round(0).astype(int)
concat.loc[concat['전용면적(㎡)']==59, '전용면적(㎡)'] = 60
concat.loc[concat['전용면적(㎡)']==84, '전용면적(㎡)'] = 85
concat.loc[concat['전용면적(㎡)']==115, '전용면적(㎡)'] = 116

In [138]:
# 시군구, 년월 등 분할할 수 있는 변수들은 세부사항 고려를 용이하게 하기 위해 모두 분할해 주겠습니다.
concat['구'] = concat['시군구'].map(lambda x : x.split()[1])
concat['동'] = concat['시군구'].map(lambda x : x.split()[2])
del concat['시군구']

In [139]:
concat['등기신청일자'] = concat['등기신청일자'].replace(' ', np.nan)
concat['거래유형'] = concat['거래유형'].replace('-', np.nan)
concat['중개사소재지'] = concat['중개사소재지'].replace('-', np.nan)

In [140]:
# concat 칼럼별 결측치 비율 
missing_values = concat.isnull().sum()  # 칼럼별 결측치 수 
total_rows = concat.shape[0]  # 전체 행 수 
missing_ratio = (missing_values / total_rows) * 100
missing_ratio_sorted = missing_ratio.sort_values(ascending=False)
missing_ratio_df = pd.DataFrame({'Missing Ratio (%)': missing_ratio_sorted})
missing_ratio_df

,Missing Ratio (%)
k-135㎡초과,99.970836
해제사유발생일,99.450844
k-등록일자,98.962143
등기신청일자,98.508724
중개사소재지,96.624306
거래유형,96.308552
단지소개기존clob,93.871433
k-홈페이지,89.843843
고용보험관리번호,81.620592
세대전기계약방법,78.485392


In [141]:
# 결측치 90% 이상 칼럼 제거 
over_90_na_ratio_col = missing_ratio_df[missing_ratio_df['Missing Ratio (%)'] >= 90].index
over_90_na_ratio_col = over_90_na_ratio_col.drop('k-135㎡초과') # 'k-135㎡초과' 칼럼은 drop하지않음
concat = concat.drop(columns=over_90_na_ratio_col)

In [142]:
# 불필요한 칼럼 제거 
cols_to_drop = ['k-홈페이지', '고용보험관리번호', 'k-팩스번호', 'k-전화번호','도로명', '번지',
                '부번','좌표X', '좌표Y', 'k-전체동수', '주차대수', '건축면적', 'k-관리비부과면적', 
                'k-주거전용면적','k-전용면적별세대현황(60㎡이하)','k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하',
                'k-수정일자', 'k-사용검사일-사용승인일', '단지신청일', '단지승인일', '계약일', 'k-135㎡초과',
                'k-관리방식', 'k-건설사(시공사)', 'k-시행사', 'k-난방방식', 'k-단지분류(아파트,주상복합등등)', 'k-세대타입(분양형태)', 'k-복도유형', 
              '관리비 업로드', '사용허가여부' , '경비비관리형태', '세대전기계약방법', '청소비관리형태', '건축년도', '기타/의무/임대/임의=1/2/3/4'
               ]
concat = concat.drop(columns=cols_to_drop)

In [143]:
concat.columns

Index(['본번', '아파트명', '전용면적(㎡)', '계약년월', '층', 'k-전체세대수', 'k-연면적', 'target',
       'is_test', 'rate', '구', '동'],
      dtype='object')

In [144]:
concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128094 entries, 0 to 1128093
Data columns (total 12 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   본번       1128019 non-null  float64
 1   아파트명     1125958 non-null  object 
 2   전용면적(㎡)  1128094 non-null  int32  
 3   계약년월     1128094 non-null  int64  
 4   층        1128094 non-null  int64  
 5   k-전체세대수  251969 non-null   float64
 6   k-연면적    251969 non-null   float64
 7   target   1118822 non-null  float64
 8   is_test  1128094 non-null  int64  
 9   rate     1128094 non-null  float64
 10  구        1128094 non-null  object 
 11  동        1128094 non-null  object 
dtypes: float64(5), int32(1), int64(3), object(3)
memory usage: 99.0+ MB


In [147]:
# '동'별 'target' 평균
target_mean_by_dong = concat.groupby('동')['target'].mean()
target_mean_by_dong_sorted = target_mean_by_dong.sort_values(ascending=False)
target_mean_by_dong_sorted

동
신문로2가     329833.333333
장충동1가     255131.818182
압구정동      203222.949968
용산동5가     188139.910569
홍파동       170336.419753
              ...      
영등포동4가     13926.136364
미근동        13084.848485
영등포동3가     11530.827068
안암동5가      11300.000000
장충동2가       8530.128205
Name: target, Length: 337, dtype: float64

In [186]:
#'luxury_apt' 칼럼 만들기 

# '계약년월'이 202301부터 202306까지인 데이터 필터링
filtered_df = concat[(concat['계약년월'] >= 202301) & (concat['계약년월'] <= 202306)]

# '아파트명'별 'target'의 평균값 계산
mean_target_by_apt = filtered_df.groupby('아파트명')['target'].mean()

# 평균값이 200,000 이상인 '아파트명' 추출
luxury_apt_names = mean_target_by_apt[mean_target_by_apt >= 200000].index

# 'luxury_apt' 열 추가
concat['luxury_apt'] = concat['아파트명'].apply(lambda x: 1 if x in luxury_apt_names else 0)

# 결과 출력
concat.head()

,본번,아파트명,전용면적(㎡),계약년월,층,k-전체세대수,k-연면적,target,is_test,rate,구,동,luxury_apt
0,658.0,개포6차우성,80,201712,3,270.0,22637.0,124000.0,0,3.42,강남구,개포동,0
1,658.0,개포6차우성,80,201712,4,270.0,22637.0,123500.0,0,3.42,강남구,개포동,0
2,658.0,개포6차우성,55,201712,5,270.0,22637.0,91500.0,0,3.42,강남구,개포동,0
3,658.0,개포6차우성,80,201801,4,270.0,22637.0,130000.0,0,3.47,강남구,개포동,0
4,658.0,개포6차우성,80,201801,2,270.0,22637.0,117000.0,0,3.47,강남구,개포동,0


In [182]:
concat[concat['luxury_apt']==1]

,본번,아파트명,전용면적(㎡),계약년월,층,k-전체세대수,k-연면적,target,is_test,rate,구,동,luxury_apt
204,187.0,개포주공5단지,74,201710,8,940.0,93810.0,129000.0,0,3.33,강남구,개포동,1
205,187.0,개포주공5단지,54,201710,7,940.0,93810.0,105500.0,0,3.33,강남구,개포동,1
206,187.0,개포주공5단지,74,201710,3,940.0,93810.0,130000.0,0,3.33,강남구,개포동,1
207,187.0,개포주공5단지,74,201710,7,940.0,93810.0,130500.0,0,3.33,강남구,개포동,1
208,187.0,개포주공5단지,74,201710,2,940.0,93810.0,123800.0,0,3.33,강남구,개포동,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1127707,208.0,남산롯데캐슬아이리스,103,202307,19,386.0,99539.0,NaN,1,4.28,중구,회현동1가,1
1127708,208.0,남산롯데캐슬아이리스,45,202308,22,386.0,99539.0,NaN,1,4.31,중구,회현동1가,1
1127709,208.0,남산롯데캐슬아이리스,34,202308,24,386.0,99539.0,NaN,1,4.31,중구,회현동1가,1
1127710,208.0,남산롯데캐슬아이리스,187,202309,6,386.0,99539.0,NaN,1,4.35,중구,회현동1가,1


In [185]:
concat_filtered = concat[(concat['아파트명'] == '삼풍') & (concat['계약년월'].between(202301, 202306))]
concat_filtered

,본번,아파트명,전용면적(㎡),계약년월,층,k-전체세대수,k-연면적,target,is_test,rate,구,동,luxury_apt
215558,1685.0,삼풍,79,202301,9,NaN,NaN,204000.0,0,4.58,서초구,서초동,1
215559,1685.0,삼풍,79,202303,12,NaN,NaN,220000.0,0,4.40,서초구,서초동,1
215560,1685.0,삼풍,79,202303,12,NaN,NaN,220000.0,0,4.40,서초구,서초동,1
215561,1685.0,삼풍,79,202303,12,NaN,NaN,204000.0,0,4.40,서초구,서초동,1
215562,1685.0,삼풍,79,202304,10,NaN,NaN,239000.0,0,4.24,서초구,서초동,1
215563,1685.0,삼풍,79,202304,11,NaN,NaN,210000.0,0,4.24,서초구,서초동,1
215564,1685.0,삼풍,131,202304,14,NaN,NaN,295000.0,0,4.24,서초구,서초동,1
215565,1685.0,삼풍,79,202304,15,NaN,NaN,212000.0,0,4.24,서초구,서초동,1
215566,1685.0,삼풍,79,202304,13,NaN,NaN,210000.0,0,4.24,서초구,서초동,1
215567,1685.0,삼풍,79,202305,6,NaN,NaN,212000.0,0,4.21,서초구,서초동,1


In [173]:
concat.head(100)

,본번,아파트명,전용면적(㎡),계약년월,층,k-전체세대수,k-연면적,target,is_test,rate,구,동,luxury_apt
0,658.0,개포6차우성,80,201712,3,270.0,22637.0,124000.0,0,3.42,강남구,개포동,NaN
1,658.0,개포6차우성,80,201712,4,270.0,22637.0,123500.0,0,3.42,강남구,개포동,NaN
2,658.0,개포6차우성,55,201712,5,270.0,22637.0,91500.0,0,3.42,강남구,개포동,NaN
3,658.0,개포6차우성,80,201801,4,270.0,22637.0,130000.0,0,3.47,강남구,개포동,NaN
4,658.0,개포6차우성,80,201801,2,270.0,22637.0,117000.0,0,3.47,강남구,개포동,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,141.0,개포주공1단지,42,201712,4,5040.0,240629.0,118000.0,0,3.42,강남구,개포동,NaN
96,141.0,개포주공1단지,43,201712,5,5040.0,240629.0,120000.0,0,3.42,강남구,개포동,NaN
97,141.0,개포주공1단지,58,201712,3,5040.0,240629.0,160000.0,0,3.42,강남구,개포동,NaN
98,141.0,개포주공1단지,36,201712,3,5040.0,240629.0,115000.0,0,3.42,강남구,개포동,NaN


In [87]:
# unique_id 만들기
concat['전용면적(㎡)'] = concat['전용면적(㎡)'].astype(str)
concat['unique_id'] = concat['동'] + concat['아파트명'] +  concat['전용면적(㎡)']
concat = concat.drop(columns=['동', '아파트명'])
concat['전용면적(㎡)'] = concat['전용면적(㎡)'].astype(int)

KeyError: '동'

In [88]:
# 강남여부 칼럼 만들기 

all = list(concat['구'].unique())
gangnam = ['강서구', '영등포구', '동작구', '서초구', '강남구', '송파구', '강동구']
gangbuk = [x for x in all if x not in gangnam]

is_gangnam = []
for x in concat['구'].tolist() :
  if x in gangnam :
    is_gangnam.append(1)
  else :
    is_gangnam.append(0)

# 파생변수를 하나 만릅니다.
concat['강남여부'] = is_gangnam

In [89]:
# target 값의 상위 5% 칼럼 만들기 

threshold = concat['target'].quantile(0.95)

# 상위 5%의 'target' 값을 가지는 행들의 'unique_id'을 새로운 리스트에 저장
top_5_pct = concat.loc[concat['target'] > threshold, 'unique_id'].unique().tolist()

# '아파트명'이 상위 5%에 해당하는 리스트에 있는지 여부를 나타내는 새로운 칼럼 생성
concat['top_5_pct'] = concat['unique_id'].isin(top_5_pct).astype(int)

concat['top_5_pct']

0          1
1          1
2          1
3          1
4          1
          ..
1128089    0
1128090    0
1128091    0
1128092    0
1128093    0
Name: top_5_pct, Length: 1128094, dtype: int32

In [90]:
# # '아파트나이=계약년월-건축년도' 칼럼 생성, 건축년도 칼럼 제거 
# concat['계약년'] = concat['계약년월'] // 100
# concat['apt_age'] = concat['계약년'] - concat['건축년도']
# concat = concat.drop(columns=['계약년', '건축년도'])
# concat['apt_age']

In [91]:
concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128094 entries, 0 to 1128093
Data columns (total 13 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   본번         1128019 non-null  float64
 1   전용면적(㎡)    1128094 non-null  int32  
 2   계약년월       1128094 non-null  int64  
 3   층          1128094 non-null  int64  
 4   k-전체세대수    251969 non-null   float64
 5   k-연면적      251969 non-null   float64
 6   target     1118822 non-null  float64
 7   is_test    1128094 non-null  int64  
 8   rate       1128094 non-null  float64
 9   구          1128094 non-null  object 
 10  unique_id  1125958 non-null  object 
 11  강남여부       1128094 non-null  int64  
 12  top_5_pct  1128094 non-null  int32  
dtypes: float64(5), int32(2), int64(4), object(2)
memory usage: 103.3+ MB


In [92]:
# 'target'과 rate의 상관관계 구하기 

target_counts = concat.groupby('unique_id')['target'].count()
most_target_unique_id = target_counts.idxmax()
most_target_unique_id

'신천동파크리오85'

In [93]:
target_counts.sort_values()

unique_id
영등포동라르파크31                0
북아현동힐스테이트신촌56             0
오금동에스아이팰리스송파올림픽공원28       0
홍익동홍익한신79                 0
당산동5가당산센트럴아이파크47          0
                       ... 
잠실동잠실엘스85              1932
중계동주공245               1934
창동주공17단지36             2056
도봉동한신85                2148
신천동파크리오85              2461
Name: target, Length: 28103, dtype: int64

In [94]:
concat_도봉동한신 = concat[concat['unique_id']=='도봉동한신85'][['계약년월', 'target']].sort_values(by='계약년월')

# 전체 '계약년월' 범위 생성 (200701부터 202306까지)
start_date = pd.to_datetime('2007-01-01')
end_date = pd.to_datetime('2023-06-30')
all_dates = pd.date_range(start=start_date, end=end_date, freq='M')
all_year_months = pd.DataFrame({'계약년월': all_dates.strftime('%Y%m').astype(int)})

# '계약년월'이 같은 행들의 'target' 값의 평균 계산
concat_도봉동한신_mean = concat_도봉동한신.groupby('계약년월')['target'].mean().reset_index()

# 새로운 데이터프레임 생성
concat_도봉동한신_interpolated = pd.merge(all_year_months, concat_도봉동한신_mean, on='계약년월', how='left')

# 선형 보간을 사용하여 누락된 'target' 값 채우기
concat_도봉동한신_interpolated['target'] = concat_도봉동한신_interpolated['target'].interpolate(method='linear')

# rate 데이터프레임의 'Date' 열 이름을 '계약년월'로 변경
rate = rate.rename(columns={'Date': '계약년월'})

# concat_도봉동한신_interpolated와 rate 데이터프레임을 '계약년월'을 기준으로 merge
merged_df = pd.merge(concat_도봉동한신_interpolated, rate, on='계약년월', how='left')

merged_df

,계약년월,target,rate
0,200701,25850.000000,6.11
1,200702,27940.000000,6.18
2,200703,25300.000000,6.20
3,200704,26157.142857,6.13
4,200705,26550.000000,6.17
...,...,...,...
193,202302,50026.666667,4.56
194,202303,52083.333333,4.40
195,202304,52900.000000,4.24
196,202305,53650.000000,4.21


In [95]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# rate 데이터프레임의 'Date' 열 이름을 '계약년월'로 변경
rate = rate.rename(columns={'Date': '계약년월'})

# concat_도봉동한신_interpolated와 rate 데이터프레임을 '계약년월'을 기준으로 merge
merged_df = pd.merge(concat_도봉동한신_interpolated, rate, on='계약년월', how='left')

# 'rate'와 'target' 열 선택
X = merged_df[['rate']]
y = merged_df['target']

# 정규화 적용
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

# 선형 회귀 모델 초기화
model = LinearRegression()

# 모델 학습
model.fit(X_scaled, y_scaled)

# 회귀 계수와 절편 출력
print('회귀 계수:', model.coef_)
print('절편:', model.intercept_)

회귀 계수: [[-0.33114759]]
절편: [2.3296498e-17]


In [96]:
concat.columns

Index(['본번', '전용면적(㎡)', '계약년월', '층', 'k-전체세대수', 'k-연면적', 'target', 'is_test',
       'rate', '구', 'unique_id', '강남여부', 'top_5_pct'],
      dtype='object')

In [97]:
# # train set과 test set에 있는 'unique_id'를 추출합니다.
# train_unique_ids = set(is_2023_true_train['unique_id'])
# test_unique_ids = set(is_2023_true_test['unique_id'])

# # test set에는 있지만 train set에는 없는 'unique_id'를 찾습니다.
# unique_ids_only_in_test = test_unique_ids - train_unique_ids

# # 결과를 출력합니다.
# print("Test set에만 있는 unique_id:")
# print(unique_ids_only_in_test)


In [98]:
# concat 데이터프레임 분리 

# 'unique_id'로 그룹화하여 각 그룹의 train set 계약년월 중에 2023년 1월 이후인 데이터가 하나라도 있는 경우 해당 'unique_id'를 선택합니다.
after_2023_ids = concat[concat['is_test'] == 0].groupby('unique_id').filter(lambda x: (x['계약년월'] >= 202301).any())

# after_2023_ids에 해당하는 데이터만 선택하여 after_2023_data 데이터프레임에 포함시킵니다.
after_2023_data = concat[concat['unique_id'].isin(after_2023_ids['unique_id'])]

# 남은 데이터는 나머지 데이터프레임에 포함시킵니다.
before_2023_data = concat[~concat['unique_id'].isin(after_2023_ids['unique_id'])]

In [99]:
after_2023_data

,본번,전용면적(㎡),계약년월,층,k-전체세대수,k-연면적,target,is_test,rate,구,unique_id,강남여부,top_5_pct
204,187.0,74,201710,8,940.0,93810.0,129000.0,0,3.33,강남구,개포동개포주공5단지74,1,1
206,187.0,74,201710,3,940.0,93810.0,130000.0,0,3.33,강남구,개포동개포주공5단지74,1,1
207,187.0,74,201710,7,940.0,93810.0,130500.0,0,3.33,강남구,개포동개포주공5단지74,1,1
208,187.0,74,201710,2,940.0,93810.0,123800.0,0,3.33,강남구,개포동개포주공5단지74,1,1
209,187.0,61,201710,14,940.0,93810.0,114750.0,0,3.33,강남구,개포동개포주공5단지61,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128085,843.0,60,202309,4,796.0,81605.0,NaN,1,4.35,송파구,장지동송파파인타운9단지60,1,0
1128089,816.0,85,202307,13,1402.0,190866.0,NaN,1,4.28,중랑구,신내동신내우디안1단지85,0,0
1128090,816.0,85,202307,12,1402.0,190866.0,NaN,1,4.28,중랑구,신내동신내우디안1단지85,0,0
1128092,816.0,85,202309,18,1402.0,190866.0,NaN,1,4.35,중랑구,신내동신내우디안1단지85,0,0


# 2023년 01월 이후 계약이 있는 DF: after_2023_data

In [100]:
# 이제 다시 train과 test dataset을 분할해줍니다. 위에서 제작해 놓았던 is_test 칼럼을 이용합니다.
after_2023_train = after_2023_data.query('is_test==0')
after_2023_test = after_2023_data.query('is_test==1')

# 이제 is_test 칼럼은 drop해줍니다.
after_2023_train.drop(['is_test'], axis = 1, inplace=True)
after_2023_test.drop(['is_test'], axis = 1, inplace=True)

print(after_2023_train.shape, after_2023_test.shape)

(744313, 12) (7340, 12)


In [101]:
after_2023_train

,본번,전용면적(㎡),계약년월,층,k-전체세대수,k-연면적,target,rate,구,unique_id,강남여부,top_5_pct
204,187.0,74,201710,8,940.0,93810.0,129000.0,3.33,강남구,개포동개포주공5단지74,1,1
206,187.0,74,201710,3,940.0,93810.0,130000.0,3.33,강남구,개포동개포주공5단지74,1,1
207,187.0,74,201710,7,940.0,93810.0,130500.0,3.33,강남구,개포동개포주공5단지74,1,1
208,187.0,74,201710,2,940.0,93810.0,123800.0,3.33,강남구,개포동개포주공5단지74,1,1
209,187.0,61,201710,14,940.0,93810.0,114750.0,3.33,강남구,개포동개포주공5단지61,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1118803,382.0,85,200704,4,366.0,45515.0,27000.0,6.13,은평구,구산동갈현현대85,0,0
1118811,382.0,85,200705,9,366.0,45515.0,28000.0,6.17,은평구,구산동갈현현대85,0,0
1118815,382.0,85,200707,18,366.0,45515.0,28900.0,6.24,은평구,구산동갈현현대85,0,0
1118819,382.0,85,200708,20,366.0,45515.0,28000.0,6.38,은평구,구산동갈현현대85,0,0


In [102]:
after_2023_test

,본번,전용면적(㎡),계약년월,층,k-전체세대수,k-연면적,target,rate,구,unique_id,강남여부,top_5_pct
1118827,187.0,74,202307,8,940.0,93810.0,NaN,4.28,강남구,개포동개포주공5단지74,1,1
1118828,185.0,83,202307,11,1060.0,94817.0,NaN,4.28,강남구,개포동개포주공6단지83,1,1
1118830,185.0,53,202308,9,1060.0,94817.0,NaN,4.31,강남구,개포동개포주공6단지53,1,1
1118832,1280.0,85,202307,17,1957.0,362895.0,NaN,4.28,강남구,개포동래미안블레스티지85,1,1
1118833,1280.0,85,202307,5,1957.0,362895.0,NaN,4.28,강남구,개포동래미안블레스티지85,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1128085,843.0,60,202309,4,796.0,81605.0,NaN,4.35,송파구,장지동송파파인타운9단지60,1,0
1128089,816.0,85,202307,13,1402.0,190866.0,NaN,4.28,중랑구,신내동신내우디안1단지85,0,0
1128090,816.0,85,202307,12,1402.0,190866.0,NaN,4.28,중랑구,신내동신내우디안1단지85,0,0
1128092,816.0,85,202309,18,1402.0,190866.0,NaN,4.35,중랑구,신내동신내우디안1단지85,0,0


In [103]:
# 'unique_id'를 기준으로 그룹화하여 가장 최근의 'target' 값을 선택합니다.
latest_target = after_2023_train.groupby('unique_id')['target'].last()
latest_target


unique_id
가락동가락3차쌍용스윗닷홈101동,102동85    71000.0
가락동가락금호60                   41000.0
가락동가락금호85                   63000.0
가락동가락미륭85                   60500.0
가락동가락스타클래스116               65000.0
                             ...   
흥인동동대문와이즈캐슬16               15500.0
흥인동동대문와이즈캐슬17               14300.0
흥인동위더스하임24                  32000.0
흥인동청계천두산위브더제니스117           75905.0
흥인동청계천두산위브더제니스124           80975.0
Name: target, Length: 5523, dtype: float64

In [104]:
after_2023_train['pred'] = (
    after_2023_train.sort_values('계약년월')
    .groupby('unique_id')['target']
    .transform('last')
)

In [105]:
after_2023_train[after_2023_train['unique_id']=='개포동개포주공5단지74'].sort_values(by='계약년월')

,본번,전용면적(㎡),계약년월,층,k-전체세대수,k-연면적,target,rate,구,unique_id,강남여부,top_5_pct,pred
1076080,187.0,74,200706,5,940.0,93810.0,105000.0,6.13,강남구,개포동개포주공5단지74,1,1,245000.0
1004895,187.0,74,200711,14,940.0,93810.0,97000.0,6.59,강남구,개포동개포주공5단지74,1,1,245000.0
1004901,187.0,74,200803,1,940.0,93810.0,97000.0,6.70,강남구,개포동개포주공5단지74,1,1,245000.0
1004900,187.0,74,200803,2,940.0,93810.0,95000.0,6.70,강남구,개포동개포주공5단지74,1,1,245000.0
1004904,187.0,74,200805,6,940.0,93810.0,94000.0,6.79,강남구,개포동개포주공5단지74,1,1,245000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
293119,187.0,74,202008,1,940.0,93810.0,209500.0,2.39,강남구,개포동개포주공5단지74,1,1,245000.0
293125,187.0,74,202009,5,940.0,93810.0,215000.0,2.44,강남구,개포동개포주공5단지74,1,1,245000.0
238220,187.0,74,202106,10,940.0,93810.0,273000.0,2.74,강남구,개포동개포주공5단지74,1,1,245000.0
201010,187.0,74,202301,10,940.0,93810.0,210000.0,4.58,강남구,개포동개포주공5단지74,1,1,245000.0


In [115]:
# # 'unique_id'를 기준으로 데이터프레임을 그룹화
# grouped_train = after_2023_train.groupby('unique_id')
# grouped_test = after_2023_test.groupby('unique_id')

In [123]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

def predict_target(group):
    train_data = group[group['계약년월'] <= 202306]
    test_data = group[group['계약년월'] > 202306]
    
    if len(train_data) == 0 or len(test_data) == 0:
        return None
    
    X_train = train_data[['rate']]
    y_train = train_data['target']
    X_test = test_data[['rate']]
    
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    
    X_train_scaled = scaler_X.fit_transform(X_train)
    y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
    X_test_scaled = scaler_X.transform(X_test)
    
    model = LinearRegression()
    model.fit(X_train_scaled, y_train_scaled)
    
    y_pred_scaled = model.predict(X_test_scaled)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    
    test_data['target'] = y_pred
    
    return pd.concat([train_data, test_data])

In [124]:
# 'unique_id'별로 그룹화하여 예측 수행
after_2023_train_grouped = after_2023_train.groupby('unique_id')
after_2023_test_grouped = after_2023_test.groupby('unique_id')

predicted_train_data = after_2023_train_grouped.apply(predict_target)
predicted_test_data = after_2023_test_grouped.apply(predict_target)

In [126]:
# 예측 결과 통합
predicted_data = pd.concat([predicted_train_data, predicted_test_data])
predicted_data

""


In [300]:
# 'unique_id'별로 가장 최신의 'pred' 값을 선택
latest_pred = after_2023_train.sort_values('계약년월').groupby('unique_id')['pred'].last()

# 'after_2023_test' 데이터프레임에 'target' 값을 'unique_id'의 최신 'pred' 값으로 할당
after_2023_test['target'] = after_2023_test['unique_id'].map(latest_pred)

after_2023_test[['계약년월', 'unique_id', 'target']]

,계약년월,unique_id,target
1118827,202307,개포동개포주공5단지74,245000.0
1118828,202307,개포동개포주공6단지83,249500.0
1118830,202308,개포동개포주공6단지53,180000.0
1118832,202307,개포동래미안블레스티지85,270000.0
1118833,202307,개포동래미안블레스티지85,270000.0
...,...,...,...
1128085,202309,장지동송파파인타운9단지60,100000.0
1128089,202307,신내동신내우디안1단지85,67500.0
1128090,202307,신내동신내우디안1단지85,67500.0
1128092,202309,신내동신내우디안1단지85,67500.0


In [301]:
# 예측결과를 concat 데이터프레임의 'target' 칼럼에 넣기 
concat.loc[after_2023_test.index, 'target'] = after_2023_test['target']

# 2023년 01월 이후 계약이 없는 DF: before_2023_data

In [244]:
# 이제 다시 train과 test dataset을 분할해줍니다. 위에서 제작해 놓았던 is_test 칼럼을 이용합니다.
before_2023_train = before_2023_data.query('is_test==0')
before_2023_test = before_2023_data.query('is_test==1')

# 이제 is_test 칼럼은 drop해줍니다.
before_2023_train.drop(['is_test'], axis = 1, inplace=True)
before_2023_test.drop(['is_test'], axis = 1, inplace=True)

print(before_2023_train.shape, before_2023_test.shape)

(374509, 12) (1932, 12)


In [246]:
before_2023_train['unique_id'].nunique()

22519

In [247]:
# before_2023_data 데이터프레임에서는 'unique_id'를 사용하지 않는다
before_2023_train = before_2023_train.drop(columns='unique_id')
before_2023_test = before_2023_test.drop(columns='unique_id')

In [248]:
# 먼저, 연속형 변수와 범주형 변수를 위 info에 따라 분리해주겠습니다.
continuous_columns = []
categorical_columns = []

for column in before_2023_train.columns:
    if pd.api.types.is_numeric_dtype(before_2023_train[column]):
        continuous_columns.append(column)
    else:
        categorical_columns.append(column)

print("연속형 변수:", continuous_columns)
print("범주형 변수:", categorical_columns)

연속형 변수: ['본번', '전용면적(㎡)', '계약년월', '층', 'k-전체세대수', 'k-연면적', 'target', 'rate', '강남여부', 'top_2_pct']
범주형 변수: ['구']


In [253]:
# 아래에서 범주형 변수들을 대상으로 레이블인코딩을 진행해 주겠습니다.

# 각 변수에 대한 LabelEncoder를 저장할 딕셔너리
label_encoders = {}

# Implement Label Encoding
for col in tqdm(categorical_columns):  # categorical_columns로 수정
    lbl = LabelEncoder()

    # Label-Encoding을 fit
    lbl.fit(before_2023_train[col].astype(str))
    before_2023_train[col] = lbl.transform(before_2023_train[col].astype(str))
    label_encoders[col] = lbl  # 나중에 후처리를 위해 레이블인코더를 저장해주겠습니다.

    # Test 데이터에만 존재하는 새로 출현한 데이터를 신규 클래스로 추가해줍니다.
    unique_labels_test = pd.unique(before_2023_test[col])
    for label in unique_labels_test:
        if not isinstance(label, str):  # 숫자인 경우 문자열로 변환
            label = str(label)
        if label not in lbl.classes_:  # unseen label 데이터인 경우
            lbl.classes_ = np.append(lbl.classes_, label)  # 미처리 시 ValueError발생하니 주의하세요!

    before_2023_test[col] = lbl.transform(before_2023_test[col].astype(str))


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.26it/s]


In [254]:
assert before_2023_train.shape[1] == before_2023_test.shape[1]          # train/test dataset의 shape이 같은지 확인해주겠습니다.

In [255]:
from sklearn.metrics import mean_squared_error
import optuna
from optuna.samplers import TPESampler
from sklearn.model_selection import KFold
import numpy as np
import xgboost as xgb
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

# Target과 독립변수들을 분리해줍니다.
y = before_2023_train['target']
X = before_2023_train.drop(['target'], axis=1)

# Hold out split을 사용해 학습 데이터와 검증 데이터를 8:2 비율로 나누겠습니다.
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=2023)

def objective(trial):
    param = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'booster': 'gbtree',
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'eta': trial.suggest_loguniform('eta', 1e-4, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 1, 10),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000)
    }
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_list = []
    for train_index, val_index in kf.split(X_train):
        X_train_kf, X_val_kf = X_train.iloc[train_index], X_train.iloc[val_index]  # 수정된 부분
        y_train_kf, y_val_kf = y_train.iloc[train_index], y_train.iloc[val_index]  # 수정된 부분
        model = xgb.XGBRegressor(**param)
        model.fit(X_train_kf, y_train_kf, eval_set=[(X_val_kf, y_val_kf)], early_stopping_rounds=50, verbose=False)
        preds = model.predict(X_val_kf)
        rmse = np.sqrt(mean_squared_error(y_val_kf, preds))
        rmse_list.append(rmse)
    return np.mean(rmse_list)

# Optuna 스터디 생성
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=50)

# 최적의 하이퍼파라미터 출력
best_params = study.best_trial.params
print("Best parameters:", best_params)

[I 2024-04-01 04:42:30,883] A new study created in memory with name: no-name-be9586c1-c2a1-4f7a-88de-f083caa9fe35
[I 2024-04-01 04:44:53,870] Trial 0 finished with value: 1639.1586063287027 and parameters: {'lambda': 9.915644566638385e-06, 'alpha': 0.40338008326003827, 'max_depth': 8, 'eta': 0.024810409748678097, 'gamma': 1.77071686435378e-07, 'colsample_bytree': 0.2403950683025824, 'subsample': 0.15227525095137953, 'min_child_weight': 8.795585311974417, 'n_estimators': 641}. Best is trial 0 with value: 1639.1586063287027.
[I 2024-04-01 04:47:31,981] Trial 1 finished with value: 1523.408872691798 and parameters: {'lambda': 0.004619347374377372, 'alpha': 1.4610865886287176e-08, 'max_depth': 9, 'eta': 0.21368329072358744, 'gamma': 4.997040685255803e-07, 'colsample_bytree': 0.26364247048639056, 'subsample': 0.2650640588680905, 'min_child_weight': 3.7381801866358395, 'n_estimators': 572}. Best is trial 1 with value: 1523.408872691798.
[I 2024-04-01 04:49:02,354] Trial 2 finished with value

KeyboardInterrupt: 

In [ ]:
best_model = XGBRegressor(**best_params, random_seed=1)
best_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=None)
pred = best_model.predict(X_val)

In [ ]:
# 평가
mse = mean_squared_error(y_val, pred)
rmse = np.sqrt(mse)
rmse

In [ ]:
importances = pd.Series(best_model.feature_importances_, index=list(X_train.columns))
importances = importances.sort_values(ascending=False)

plt.figure(figsize=(10,8))
plt.title("Feature Importances")
sns.barplot(x=importances, y=importances.index)

In [ ]:
# 학습된 모델을 저장합니다. Pickle 라이브러리를 이용하겠습니다.
with open('saved_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

In [ ]:
# 저장된 모델을 불러옵니다.
with open('saved_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
%%time
X_test = dt_test.drop(['target'], axis=1)

# Test dataset에 대한 inference를 진행합니다.
pred = model.predict(X_test)

In [ ]:
concat.loc[before_2023_test.index, 'target'] = pred

In [ ]:
concat[concat['is_test']==1]['target'].to_csv('output_MA_2.csv', header=True, index=False)